https://www.earthdatascience.org/courses/use-data-open-source-python/hierarchical-data-formats-hdf/open-MODIS-hdf4-files-python/


In [1]:
# Import packages
import os
import re  # regular expressions
import warnings
import matplotlib.pyplot as plt
import numpy as np
import numpy.ma as ma
import rasterio as rio
from rasterio.plot import plotting_extent
import geopandas as gpd
import earthpy as et
import earthpy.plot as ep
import earthpy.spatial as es
import earthpy.mask as em

warnings.simplefilter('ignore')

# Set working directory
os.chdir(os.path.join(et.io.HOME, 'nyu', 'rbda_project_data'))


In [2]:
# Create a path to the pre-fire MODIS h4 data
thermal_anomalies_path = os.path.join("precipitation_11_24",
                             "3B-HHR.MS.MRG.3IMERG.20200731-S233000-E235959.1410.V06B.HDF5")


In [3]:
# View dataset metadata
with rio.open(thermal_anomalies_path) as dataset:
    print(dataset)
    hdf4_meta = dataset.meta

# Notice that there are metadata at the highest level of the file
hdf4_meta


<open DatasetReader name='precipitation_11_24/3B-HHR.MS.MRG.3IMERG.20200731-S233000-E235959.1410.V06B.HDF5' mode='r'>


{'driver': 'HDF5',
 'dtype': 'float_',
 'nodata': None,
 'width': 512,
 'height': 512,
 'count': 0,
 'crs': None,
 'transform': Affine(1.0, 0.0, 0.0,
        0.0, 1.0, 0.0)}

In [7]:
# Print all of the subdatasets in the data
with rio.open(thermal_anomalies_path) as dataset:
    crs = dataset.read_crs()
    for name in dataset.subdatasets:
        print(name)


HDF5:precipitation_11_24/3B-HHR.MS.MRG.3IMERG.20200731-S233000-E235959.1410.V06B.HDF5://Grid/HQobservationTime
HDF5:precipitation_11_24/3B-HHR.MS.MRG.3IMERG.20200731-S233000-E235959.1410.V06B.HDF5://Grid/precipitationUncal
HDF5:precipitation_11_24/3B-HHR.MS.MRG.3IMERG.20200731-S233000-E235959.1410.V06B.HDF5://Grid/probabilityLiquidPrecipitation
HDF5:precipitation_11_24/3B-HHR.MS.MRG.3IMERG.20200731-S233000-E235959.1410.V06B.HDF5://Grid/randomError
HDF5:precipitation_11_24/3B-HHR.MS.MRG.3IMERG.20200731-S233000-E235959.1410.V06B.HDF5://Grid/time_bnds
HDF5:precipitation_11_24/3B-HHR.MS.MRG.3IMERG.20200731-S233000-E235959.1410.V06B.HDF5://Grid/HQprecipSource
HDF5:precipitation_11_24/3B-HHR.MS.MRG.3IMERG.20200731-S233000-E235959.1410.V06B.HDF5://Grid/HQprecipitation
HDF5:precipitation_11_24/3B-HHR.MS.MRG.3IMERG.20200731-S233000-E235959.1410.V06B.HDF5://Grid/IRkalmanFilterWeight
HDF5:precipitation_11_24/3B-HHR.MS.MRG.3IMERG.20200731-S233000-E235959.1410.V06B.HDF5://Grid/IRprecipitation
HDF5:

In [8]:
# Create empty list to append arrays (of band data)
all_bands = []

# Open the pre-fire HDF4 file
with rio.open(thermal_anomalies_path) as dataset:
    
    # Loop through each subdataset in HDF4 file
    for name in dataset.subdatasets:

        # Open the band subdataset
        with rio.open(name) as subdataset:
            modis_meta = subdataset.profile

            # Read band data as a 2 dim arr and append to list
            all_bands.append(subdataset.read(1))

# Stack pre-fire reflectance bands
thermal_anomalies_modis = np.stack(all_bands)
thermal_anomalies_modis.shape


ValueError: all input arrays must have the same shape

In [ ]:
ep.plot_bands(thermal_anomalies_modis,
              scale=False)
plt.show()


In [ ]:
# View entire metadata object 
# for the last MODIS band processed in loop
modis_meta


In [ ]:
# View just the nodata value
modis_meta["nodata"]


In [ ]:
# Mask no data values
thermal_anomalies_modis = ma.masked_where(
    thermal_anomalies_modis == modis_meta["nodata"], thermal_anomalies_modis)


In [ ]:
# ep.plot_bands(thermal_anomalies_modis,
#               scale=False,
#               figsize=(10, 7))
# plt.show()


In [ ]:
# Plot MODIS RGB
ep.plot_rgb(thermal_anomalies_modis,
#             rgb=[0, 1, 3],
            title='RGB Image of MODIS Data (thermal anomalies)',
            stretch=True,
            figsize=(7, 7))

plt.show()
